In [2]:
# Imports - getting relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter 
import numpy as np
from operator import itemgetter
from scipy import linalg
from os import *
from os.path import isfile, join
import time
from datetime import date, timedelta
from pylab import *

from IPython.display import Image 
# API access to plotting tools
plotly.tools.set_credentials_file(username='frksteenhoff2', api_key ='duu8hsfRmuI5rF2EU8o5')
import plotly.plotly as py
import plotly.graph_objs as go
import re
from fractions import Fraction

# Setting the colors for the different plots, 
# such that all plots get the same colors
homeGreen = 'rgb(0, 204, 0)'         # bright green 
hedeBlue  = 'rgb(0, 0, 255)'         # blue
pieGreen  = homeGreen                # bright green
pieOrange = 'rgb(246, 214, 56)'      # orange
pieRed    = 'rgb(204, 0, 0)'         # red
ticksAxes = 'rgb(107, 107, 107)'     # axes color - grey
bgBorder  = 'rgba(255, 255, 255, 0)' # white

## Read in correct data
From both Netatmo and Wireless

In [532]:
all_files = [] # All files
co2_files = [] # CO2 data from netatmo files
wir_files = [] # Wireless - when-are-people-home-files
# Change directory to 
#chdir(processpath)

all_files += [file for file in listdir('.') if file.endswith('.xlsx')]

for files in all_files:
    print files 
    if 'PIRReed' in files:
        wir_files.append(files)
    elif 'netatmo' in files:
        co2_files.append(files)

#print "Files: ", all_files
#print "\nFiles in all: ", len(all_files)

he201-PIRReed_entrance.xlsx
He211-PIRReed_entrance.xlsx
netatmo_He211 fBedroom downstairs.xlsx
testfile.xlsx
testfile_co2.xlsx
~$netatmo_He211 fBedroom downstairs.xlsx
~$testfile.xlsx
~$testfile_co2.xlsx


In [539]:
wir_files[0][:6]

'he201-'

In [ ]:
firstOC = True   # only holds until first open/close value has been found
doorOpen = False # check whether door has been closed or opened last
home_interval = pd.DataFrame()

# ----------------------------------------------------------------- #
# CO2-plots for time when people are home for each room in house
# ----------------------------------------------------------------- #
for homes in wir_files:
    # read in the two fields
    alias = homes[:5]
    wir_data = pd.read_excel(homes)
    # Extract needed columns from dataset
    netatmo_min = wir_data[['code','submitDate','submitTime']]

    # Converting all values to time values
    new_time = []
    for row in range(0, len(netatmo_min)):
        td1 = datetime.timedelta(hours=netatmo_min.iloc[row,2])
        new_time.append((datetime.datetime(2017,1,1)+td1).strftime("%H:%M:%S"))
    # If you want to check the values
    #new_time
    # Overwriting old time values
    netatmo_min['submitTime'] = new_time
    
# ----------------------------------------------------------------- #
# Find intervals where people are at home 
# ----------------------------------------------------------------- #
    for row in range(0,len(netatmo_min)):
        # If movement occurs before door has been opened, someone is home
        if netatmo_min.iloc[row, 0] == 'open' and firstOC:
            if row != 0:
                home_interval = pd.concat([home_interval, netatmo_min.iloc[[0]], netatmo_min.iloc[[row]]])
            firstOC = False
            doorOpen = True

        # If movement occurs after door has been opened, ignore 
        elif netatmo_min.iloc[row, 0] == 'closed' and firstOC:
            home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
            firstOC = False
            doorOpen = False

        # If there has been movement while the door has been closed, someone is home
        elif netatmo_min.iloc[row, 0] == 'open' and netatmo_min.iloc[row-1, 0] == 'movement':
            home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
            doorOpen = True

        # If movement occurs after door has been opened, ignore 
        elif netatmo_min.iloc[row, 0] == 'closed' and netatmo_min.iloc[row+1, 0] != 'open':
            home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
            doorOpen = False

        # If end of file and door is closed, include sequence.
        elif row == len(netatmo_min)-1 and doorOpen == False:
            end = netatmo_min.iloc[[row]]
            end['code'] = 'end'
            home_interval = pd.concat([home_interval, end])

    # IF movement between closed and open, add close/open sequence to new dataframe.
    #    '''elif netatmo_min.iloc[row, 0] == 'open' and netatmo_min.iloc[row+1, 0] == 'movement' and doorOpen == False:
    #        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
    #        doorOpen = True
    #
    #    elif netatmo_min.iloc[row, 0] == 'closed' and netatmo_min.iloc[row-1, 0] == 'movement' and doorOpen:
    #        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
    #        doorOpen = False'''
# ----------------------------------------------------------------- #    
# Find rooms to calculate CO2-levels in
# ----------------------------------------------------------------- #
    for rooms in co2_files:
        if alias in rooms: 
            current_rooms.append(rooms)
# ----------------------------------------------------------------- #    
# For each room, find and combine co2 and home info
# ----------------------------------------------------------------- #
    # read in the two fields
    for rooms in current_rooms:
        co2_data = pd.read_excel(rooms)
        co2_data = co2_data[['Timezone : Europe/Stockholm', 'CO2']]
        co2_data['submitTime'] = pd.to_datetime(co2_data['Timezone : Europe/Stockholm']).dt.time
        co2_data['submitDate'] = pd.to_datetime(co2_data['Timezone : Europe/Stockholm']).dt.date
        co2_data[:10]

        basic_frame = pd.DataFrame({'code': [0],
                                    'CO2': [0],
                                   'submitDate': [0],
                                   'submitTime': [0]})
        new = pd.DataFrame()
        new = basic_frame

        for row in range(0,len(co2_data)):
            basic_frame['submitDate'] = co2_data.iloc[row, 3]
            basic_frame['submitTime'] = co2_data.iloc[row, 2]
            basic_frame['CO2'] = co2_data.iloc[row, 1]

            new = pd.concat([new, basic_frame])
        #td1 = datetime.timedelta(hours=basic_frame.iloc[0,2])
        #(datetime.datetime(2017,1,1)+td1).strftime("%H:%M:%S")
        newf = pd.concat([home_interval, new])
        newf = newf.sort_values(by=['submitDate','submitTime'])
        # Save info to file
        writer = pd.ExcelWriter('testfile.xlsx', engine='xlsxwriter')
        newf.to_excel(writer)
        writer.save()        
        
        closed = False
        finalco2 = pd.DataFrame(columns=newf.columns)
        endOfTimeHome = False
        
# ----------------------------------------------------------------- #        
# Get final CO2 data for plotting
# ----------------------------------------------------------------- #
        for row in range(0,len(newf)):
            # Only as long as there are still values when people are home
            if newf.iloc[row,1] != 'end' and endOfTimeHome == False:
                # check whether values are between bounds
                if newf.iloc[row, 1] != long(0): # If value
                    if newf.iloc[row, 1] == 'movement' or newf.iloc[row, 1] == 'closed':
                        finalco2 = pd.concat([finalco2, newf.iloc[[row]]])
                        closed = True
                    elif newf.iloc[row, 1] == 'open':
                        finalco2 = pd.concat([finalco2, newf.iloc[[row]]])
                        closed = False
                elif closed and newf.iloc[row, 1] == long(0):
                    finalco2 = pd.concat([finalco2, newf.iloc[[row]]])
            else:
                # Skip all co2 values when people not at home
                endOfTimeHome = True
        finalco2 = finalco2[finalco2.loc['CO2'] != 0.0]
# ----------------------------------------------------------------- #        
# Save plot values
# ----------------------------------------------------------------- #
        finalco2 = finalco2.loc[finalco2['CO2'] != 0.0]
        val = []
        # Find CO2-values between boundaries!
        perc_u = finalco2.loc[finalco2['CO2'] < 1000]
        perc_b = finalco2.loc[finalco2['CO2'] >= 1000]
        prcb2  = perc_b.loc[perc_b['CO2'] < 1500]
        perc_o = finalco2.loc[finalco2['CO2'] >= 1500]
        print len(perc_u), len(prcb2), len(perc_o), len(finalco2)

        val.append(float(len(perc_u)) / float(len(finalco2)) * 100)
        val.append(float(len(prcb2))  / float(len(finalco2)) * 100)
        val.append(float(len(perc_o)) / float(len(finalco2)) * 100)
        val

# ----------------------------------------------------------------- #
# Save plot to proper location
# ----------------------------------------------------------------- #
        plotType = '-fa'          # type: fresh air
        # week number and home alias from earlier variables
        # Plot of air 
        fig = {
            'data': [{'labels': ['Indenfor anbefaling', 'Lidt over anbefaling', 'Over anbefaling'],
                      'values': val,
                      'type': 'pie',
                      'marker': {'colors': [pieGreen,
                                            pieOrange,
                                            pieRed]},
                      'textinfo': 'none'}],              
            'layout': { 'autosize': False,
                        'width': 350,
                        'height': 350,
                        "paper_bgcolor": "rgba(0, 0, 0, 0)",
                        "plot_bgcolor": "rgba(0, 0, 0, 0)",
                        'showlegend': False}
             }

        # Save to folder
        fullPathToPlot = alias + 'co2' + plotType + ".png"
        py.image.save_as(fig, filename=fullPathToPlot)
        #Image(fullPathToPlot) # Display static image
        # Reset plotting array
        val = []

In [540]:
co2_files in 

['netatmo_He211 fBedroom downstairs.xlsx',
 '~$netatmo_He211 fBedroom downstairs.xlsx']

OTHER!!


In [294]:
# read in the two fields
netatmo_data = pd.read_excel('he201-PIRReed_entrance.xlsx')

In [290]:
#netatmo_data['time_'] = time-strftime(netatmo_data['time_']).dt.
netatmo_min = netatmo_data[['code','submitDate','submitTime']]

# Converting all values to time
new_time = [] 
for row in range(0, len(netatmo_min)):
    td1 = datetime.timedelta(hours=netatmo_min.iloc[row,2])
    new_time.append((datetime.datetime(2017,1,1)+td1).strftime("%H:%M:%S"))
# If you want to check the values
#new_time

TypeError: unsupported type for timedelta hours component: datetime.time

## When are the people at home?
When there has been movement after the door has closed.

In [467]:
firstOC = True   # only holds until first open/close value has been found
doorOpen = False # check whether door has been closed or opened last
home_interval = pd.DataFrame()

for row in range(0,len(netatmo_min)):
    # If movement occurs before door has been opened, someone is home
    if netatmo_min.iloc[row, 0] == 'open' and firstOC:
        if row != 0:
            home_interval = pd.concat([home_interval, netatmo_min.iloc[[0]], netatmo_min.iloc[[row]]])
        firstOC = False
        doorOpen = True
        
    # If movement occurs after door has been opened, ignore 
    elif netatmo_min.iloc[row, 0] == 'closed' and firstOC:
        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
        firstOC = False
        doorOpen = False
        
    # If there has been movement while the door has been closed, someone is home
    elif netatmo_min.iloc[row, 0] == 'open' and netatmo_min.iloc[row-1, 0] == 'movement':
        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
        doorOpen = True

    # If movement occurs after door has been opened, ignore 
    elif netatmo_min.iloc[row, 0] == 'closed' and netatmo_min.iloc[row+1, 0] != 'open':
        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
        doorOpen = False
    
    # If end of file and door is closed, include sequence.
    elif row == len(netatmo_min)-1 and doorOpen == False:
        end = netatmo_min.iloc[[row]]
        end['code'] = 'end'
        home_interval = pd.concat([home_interval, end])

# IF movement between closed and open, add close/open sequence to new dataframe.

    '''elif netatmo_min.iloc[row, 0] == 'open' and netatmo_min.iloc[row+1, 0] == 'movement' and doorOpen == False:
        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
        doorOpen = True
    
    elif netatmo_min.iloc[row, 0] == 'closed' and netatmo_min.iloc[row-1, 0] == 'movement' and doorOpen:
        home_interval = pd.concat([home_interval, netatmo_min.iloc[[row]]])
        doorOpen = False'''
    

C:\Users\frksteenhoff\Anaconda2\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [535]:
#home_interval

## CO2 levels when people are home

In [333]:
# read in the two fields
co2_data = pd.read_excel('netatmo_He211 fBedroom downstairs.xlsx')
co2_data = co2_data[['Timezone : Europe/Stockholm', 'CO2']]
co2_data['submitTime'] = pd.to_datetime(co2_data['Timezone : Europe/Stockholm']).dt.time
co2_data['submitDate'] = pd.to_datetime(co2_data['Timezone : Europe/Stockholm']).dt.date
co2_data[:10]

,Timezone : Europe/Stockholm,CO2,submitTime,submitDate
2,2017/03/27 00:01:19,1913.0,00:01:19,2017-03-27
3,2017/03/27 00:06:25,1916.0,00:06:25,2017-03-27
4,2017/03/27 00:11:33,1867.0,00:11:33,2017-03-27
5,2017/03/27 00:16:41,1838.0,00:16:41,2017-03-27
6,2017/03/27 00:21:48,1905.0,00:21:48,2017-03-27
7,2017/03/27 00:26:57,1985.0,00:26:57,2017-03-27
8,2017/03/27 00:32:04,1983.0,00:32:04,2017-03-27
9,2017/03/27 00:36:21,2006.0,00:36:21,2017-03-27
10,2017/03/27 00:40:37,2024.0,00:40:37,2017-03-27
11,2017/03/27 00:45:45,2027.0,00:45:45,2017-03-27


In [461]:
basic_frame = pd.DataFrame({'code': [0],
                            'CO2': [0],
                           'submitDate': [0],
                           'submitTime': [0]})
new = pd.DataFrame()
new = basic_frame

for row in range(0,len(co2_data)):
    basic_frame['submitDate'] = co2_data.iloc[row, 3]
    basic_frame['submitTime'] = co2_data.iloc[row, 2]
    basic_frame['CO2'] = co2_data.iloc[row, 1]
    
    new = pd.concat([new, basic_frame])
#td1 = datetime.timedelta(hours=basic_frame.iloc[0,2])
#(datetime.datetime(2017,1,1)+td1).strftime("%H:%M:%S")
basic_frame

,CO2,code,submitDate,submitTime
0,2990.0,0,2017-04-02,23:57:58


In [301]:
basic_frame['submitTime'] = pd.to_datetime(basic_frame['submitTime']).dt.time

In [469]:
newf = pd.concat([home_interval, new])
newf = newf.sort_values(by=['submitDate','submitTime'])

In [472]:
writer = pd.ExcelWriter('testfile.xlsx', engine='xlsxwriter')
newf.to_excel(writer)
writer.save()        

In [490]:
closed = False
finalco2 = pd.DataFrame(columns=newf.columns)
endOfTimeHome = False

for row in range(0,len(newf)):
    # Only as long as there are still values when people are home
    if newf.iloc[row,1] != 'end' and endOfTimeHome == False:
        # check whether values are between bounds
        if newf.iloc[row, 1] != long(0): # If value
            if newf.iloc[row, 1] == 'movement' or newf.iloc[row, 1] == 'closed':
                finalco2 = pd.concat([finalco2, newf.iloc[[row]]])
                closed = True
            elif newf.iloc[row, 1] == 'open':
                finalco2 = pd.concat([finalco2, newf.iloc[[row]]])
                closed = False
        elif closed and newf.iloc[row, 1] == long(0):
            finalco2 = pd.concat([finalco2, newf.iloc[[row]]])
    else:
        # Skip all co2 values when people not at home
        endOfTimeHome = True
finalco2 = finalco2[finalco2.loc['CO2'] != 0.0]

,CO2,code,submitDate,submitTime
22,0.0,movement,2017-03-27 00:00:00,00:10:18
0,1867.0,0,2017-03-27,00:11:33
0,1838.0,0,2017-03-27,00:16:41
0,1905.0,0,2017-03-27,00:21:48
0,1985.0,0,2017-03-27,00:26:57
0,1983.0,0,2017-03-27,00:32:04
0,2006.0,0,2017-03-27,00:36:21
0,2024.0,0,2017-03-27,00:40:37
0,2027.0,0,2017-03-27,00:45:45
0,2024.0,0,2017-03-27,00:50:52


In [504]:
finalco2 = finalco2.loc[finalco2['CO2'] != 0.0]

val = []
# Find CO2-values between boundaries!
perc_u = finalco2.loc[finalco2['CO2'] < 1000]
perc_b = finalco2.loc[finalco2['CO2'] >= 1000]
prcb2  = perc_b.loc[perc_b['CO2'] < 1500]
perc_o = finalco2.loc[finalco2['CO2'] >= 1500]
print len(perc_u), len(prcb2), len(perc_o), len(finalco2)
    
val.append(float(len(perc_u)) / float(len(finalco2)) * 100)
val.append(float(len(prcb2))  / float(len(finalco2)) * 100)
val.append(float(len(perc_o)) / float(len(finalco2)) * 100)
val


# Save plot to proper location
plotType = '-fa'          # type: fresh air
# week number and home alias from earlier variables
# Plot of air 
fig = {
    'data': [{'labels': ['Indenfor anbefaling', 'Lidt over anbefaling', 'Over anbefaling'],
              'values': val,
              'type': 'pie',
              'marker': {'colors': [pieGreen,
                                    pieOrange,
                                    pieRed]},
              'textinfo': 'none'}],              
    'layout': { 'autosize': False,
                'width': 350,
                'height': 350,
                "paper_bgcolor": "rgba(0, 0, 0, 0)",
                "plot_bgcolor": "rgba(0, 0, 0, 0)",
                'showlegend': False}
     }

# Save to folder
fullPathToPlot = 'co2' + plotType + ".png"
py.image.save_as(fig, filename=fullPathToPlot)
#Image(fullPathToPlot) # Display static image
# Reset plotting array
val = []

writer = pd.ExcelWriter('testfile_co2.xlsx', engine='xlsxwriter')
finalco2.to_excel(writer)
writer.save()        

279 361 381 1021
